In [2]:
from calculations.damage_calculator import damage_calc
from calculations.dataclasses_and_enums import OneTalents, SevenTalents

In [3]:
time_series, damage_series = damage_calc(7, 10, one_talent=OneTalents.WAYOFILLUSION, seven_talent=SevenTalents.CRUSHINGBLOWS)

278.76006637128916 CRIT
574.9426368907839 CRIT -W
784.0126866692507 AA
993.0827364477175 AA
1202.1527862261844 AA
1515.7578608938848 CRIT
1829.362935561585 CRIT -W
2055.8554894882573 AA
2282.3480434149296 AA
2508.840597341602 AA
2822.445672009302 CRIT
3136.0507466770023 CRIT -W
3362.5433006036747 AA
3589.035854530347 AA
3815.5284084570194 AA
4129.1334831247195 CRIT
4442.73855779242 CRIT -W
4669.231111719092 AA
4895.723665645764 AA
5122.216219572437 AA


In [5]:
import matplotlib.pyplot as plt